In [1]:
import numpy as np
import requests
import os
import io
import json
import passporteye
import cv2
import pytesseract
import easyocr

import matplotlib as plt
from matplotlib.pyplot import imshow

%matplotlib inline


In [2]:
path = 'E:/mypath/ DataForOCR/'
image_names = os.listdir(path)


### Использование готовых библиотек Python для распознавания документов

Мною было найдено две готовые библиотеки для распознавания документов: 
- PassportEye (https://github.com/konstantint/PassportEye) и
- paSport_eye (https://github.com/hulumulu801/paSport_eye). 

Вторая рассчитана на опереационную систему линукс, поэтому опробовала пока первую. 

In [6]:
for name in image_names: 
    image_path = path + name
    mrz = passporteye.read_mrz(image_path)
    print(name)
    print(mrz)
    print('--------------------------------')

0.jpeg
MRZ(TD2[2], 0511HPECH, , PTHWEMZOBWOTRZJIZHMZH, A, 0F0PM0)
--------------------------------
1.jpeg
None
--------------------------------
2.jpeg
MRZ(TD2[2], <<2M111IE, , M3WBIIPQH>WIO, E, 18X>06)
--------------------------------
3.jpeg
None
--------------------------------
4.png
None
--------------------------------
5.jpeg
MRZ(TD3[44], S81180842, ROMAN 7R9EVI3, MAKAROV, M, 950616)
--------------------------------
6.jpeg
None
--------------------------------
7.jpeg
MRZ(TD2[2], QKTEIBPBC, , JIOMBHVTPEHHHXAEJI, <, 0KPYFA)
--------------------------------
8.jpeg
None
--------------------------------
9.jpeg
None
--------------------------------


К сожалению, данная библиотека не рассчитана на работу с кириллицей, поэтому результаты либо отсутствуют, либо скудные и нечитаемые. 

### API

#### Free OCR API

In [27]:
api_key = 'myapikey'
url_api = 'https://api.ocr.space/parse/image'
payload ={'isOverlayRequired': True,
        'apikey': api_key,
         'language': 'rus'}

In [50]:
for name in image_names: 
    im_path = path + name
    im_read = cv2.imread(im_path)
    im_read = imutils.resize(im_read, height=1000)
    img_encode = cv2.imencode('.jpeg', im_read)[1]
    file_bytes = io.BytesIO(img_encode)
    res = requests.post(url_api, 
             files={name: file_bytes}, 
            data = payload)
    result = json.loads(res.content.decode())
    print(name)
    print(result.get('ParsedResults')[0].get('ParsedText'))
    print('--------------------------------')

0.jpeg
госсийск А Я Ф гд ЕРА ция
ПАСПОРТНО-ВИЗОВЬМ ОТДЕЛЕНИЕМ
ОВД ПРЕСНЕНСКОГО РАЙОНА
увд ЦАО ГОРОДА МОСКВЫ
ь, МУЖ.
772-112
ТРАМП
ДОНАЛЬД
джон
14.06.1946
ГОР. КУИНС
ШТАТ НЬЮ-ЙОРК
США

--------------------------------
1.jpeg
выдач.
P
123 - 005
17 . 1988
о
СС ИЙ СК Л Я ф Е д
МО УФМС РОССИИ
по мурмАнской ОБЛАСТИ
В ГОРОДЕ мончегорск
05 11 2008
ХАЛАБУДИНА
юлия
АЛЕКСЕЕВНА
ж
г. МОНЧЕГОРСК

--------------------------------
2.jpeg
РОССИЙСК А Я Ф Е ДЕР А ция
ОТДЕЛОМ ВНУТРЕННИХ ДЕЛ
ГОР . КРАСНОЗНАМЕНСК
16.10.2016
ЖЕН.
100-128
ШАПОШНИКОВА
виктория
НИКОЛАЕВНА
14.09.1985
КРАСНОЗНАМЕНСК
страничка Паспорт РФ 2
<<<Стоимость программы 10
: hackerlabnew@qmai1. сот>>>>

--------------------------------
3.jpeg
РОСС ий с КА Я Ф Е ДЕРАЦИЯ
ОТДЕЛОМ УФМС РОССИИ ПО
САМАРСКОЙ ОБЛАСТИ В ЦЕНТРАЛЬНОМ
РАЙОНЕ ГОРОДА ТОЛЬЯТТИ
08.05.2006
Д•т• вымчи
КОА
с.цу,т
муж .
112-009
ДАймонд
— дмитрий
АЛЕКСЕЕВИЧ
0$. 05.1986
ГОР . САМАРА

--------------------------------
4.png
ТП ОТДГ„ЛА УФМС России
ПО САНКТ-ПЕТЕРБУРГУ И ЛЕНИНГРАДС

#### mindee

In [27]:
im_path = path + image_names[0]
from mindee import Client, documents

mindee_client = Client(api_key="myapikey")

input_doc = mindee_client.doc_from_path(im_path)

api_response = input_doc.parse(documents.TypePassportV1)
print(api_response.document)

-----Passport data-----
Filename: 0.jpeg
Full name:
Given names:
Surname:
Country: TTO
ID Number:
Issuance date:
Birth date:
Expiry date:
MRZ 1: P<TTOCLILLE<<TILLI<<<<<<<<<<<<<<<<<<<<<<<<<<
MRZ 2:
MRZ:
----------------------


### Мой алгоритм

In [28]:
# Алгоритм находится в отдельном скрипте
from OcrScript import get_document_data

In [6]:
for name in image_names: 
    image_path = path + name
    my_result = get_document_data(image_path)
    print(name)
    print(my_result)
    print('--------------------------------')


0.jpeg
['ПАСПОРТНО-ВИЗОВЫ', 'ОТДЕЛЕНИЕМ', 'ОВД', 'ПРЕСНЕНСКОГО', 'РАЙОНА', 'УВД', 'ШАО', 'ГОРОДА', 'МОСКВЫ', '772-112', '-09', '11.201 6   .', '7', 'ТРАМП', 'ДОНАЛЬД', 'ДХОН', '14.06.1 946', 'АЗ', '', 'МУХ', '0', 'КУИНС', 'ГОР', 'ШТАТ', 'НЬЮ-ЙОРК', 'США', '45', '07', '1234 56', '45', 'А7', '123458']
--------------------------------
1.jpeg
[' 0   И Й    Я', 'Ф  4 Е  А Ц И Я', 'О', 'УФМС', 'РОССИИ', 'ПО МУРМАНСКОЙ ОБЛАСТИ', 'В ГОРОДЕ МОНЧЕГОРСК', ' ПИМЧП', '05', '11', '2008', '123', '005', 'О', 'ХАЛАБУДИНА', 'ЮЛИЯ', 'АЛЕКСЕЕВНА', '17', '10', '1988', 'Г', 'МОНЧЕГОРСК', '66', '06', '304001', 'С', '', '', '66', '06', '304001']
--------------------------------
2.jpeg
['ОТДЕЛОМ', 'ВНУТРЕННИХ', 'ДЕЛ', 'КРАСНОЗНАМЕНСК', 'ГОР', '16', '10', '2016', 'О', '100-128', 'ШАПОШНИКОВА', 'ВИКТОРИЯ', 'НИКОЛАЕВНА', '14 . 09', '1985', 'ЖВН', 'КРАСНОЗНАМЕС', 'ГОР', 'ПАСПОРТ', 'РФ', '2017', 'ПРОБНАЯ', 'СТРАНИАКА', 'СТОИМОСТЬ', 'ПРОГРАММЫ', '10', '8', ' ', '', 'А1', '76', '06', '991 745', '76', '06', '991 745']

Как можно видеть, мой алгоритм хорошо справляется далеко не со всеми документами. Особенно тяжело даются документы с низким качеством.
Для распознавания текста была использована библиотека easyocr. Также была опробована библиотека pytesseract, но в данном случае достичь приемлемого результата с этой библиотекой не получилось. 